# Grocery Store Prices Crawler

I originally did this to track beer prices near me. It can be tweaked to fetch the prices from other products aswell.

### To-do list:
- Add sections tracking (e.g. cheese, meats, flour, etc)
- Add notifications in Hyprland (notify-send)

#### Functions

In [ ]:
def procura(items:list, produto):
    """
    :param items: list, lista de produtos a se verificar
    :param produtos: DataFrame, o banco de dados que contem todos os produtos que serão verificadas
    """
    for i in items:
        if 'milka' in i.lower():
            searchfor = 1
        elif 'guinness' in i.lower():
            searchfor = 2
    
    
    if searchfor == 1:
        try:
            encontrado = 0
            for item in items:
                index = 0
                for stock in produto.Chocolate:
                    if item in stock.lower():
                        print(f'Em {produto['Data'][index]}: {stock}, {produto.Embalagem[index]}, por {produto['Custo de compra'][index]}€, preço por kg {produto['Preço por kg'][index]}€.')
                        encontrado = 1
                    index += 1

            if encontrado == 0:
                print('Não encontrei nenhum dos chocolates.')
        except:
            print('Algo deu errado. Algum erro não previsto na verificação dos chocolates.')


    elif searchfor == 2:
        try:
            encontrado = 0
            for item in items:
                index = 0
                for stock in produto.Cerveja:
                    if item in stock.lower():
                        print(f'Em {produto['Data'][index]}: {stock}, {produto.Embalagem[index]}, por {produto['Custo de compra'][index]}€, preço por litro {produto['Preço por litro'][index]}€.')
                        encontrado = 1
                    index += 1

            if encontrado == 0:
                print('Não encontrei nenhuma das cervejas.')
        except:
            print('Algo deu errado. Algum erro não previsto na verificação das cervejas.')


    else:
        print('Não sei o que fazer. Não encontrei nem Milka nem Guinness nas tuas listas. Te vira.')

The code

In [1]:
# Importing necessary libraries
#import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from datetime import datetime
from time import sleep

# Create a dict with the stores and their website
stores = {'Store':['Continente', 'Pingo Doce'],
          'Website':['https://www.continente.pt', 'https://www.pingodoce.pt']}

# Items that I want to track
my_beers = ['coruja india pale ale', 'guinness', 'franziskaner', 'estrela damm', 'erdinger', 'benediktiner']
my_choc = ['milka', 'biglicious', 'biiig'] # biglicious e biiig são as barras de 300g do Continente/Pingo Doce





In [ ]:
op = webdriver.FirefoxOptions()
#op.add_argument('--headless')
driver = webdriver.Firefox(options=op)
try:
    print('Acessando a web...')
    driver.get('https://www.continente.pt')
    print('Página carregada. Livrando cookies...')
    element = driver.find_element(By.ID, "CybotCookiebotDialogBodyLevelButtonCustomize").click()
    print('Cookies 1/2 foi')
    sleep(1)
    element = driver.find_element(By.ID, "CybotCookiebotDialogBodyButtonDecline").click()
    print('Cookies 2/2 foi. Eliminados.')
    sleep(1)
    try:
        print('Abrindo menu sanduíche')
        element = driver.find_element(By.CLASS_NAME, "navbar-toggler").click()
    except Exception as e:
        print('Se deu erros maiores, pára aqui. Se continuar é porque não tinha menu sanduíche para abrir')
        #print(f'ESSA É A MENSAGEM DO SEGUNDO TRY EXCEPT: {e}')
    print('Clicando em Produtos')
    element = driver.find_element(By.ID, "col-produtos").click()
    sleep(1)
    print('Abrindo "Bebidas e Garrafeira"')
    element = driver.find_element(By.PARTIAL_LINK_TEXT, "Bebidas e Garrafeira").click()
    sleep(1)
    print('Abrindo "Cervejas e Sidras"')
    element = driver.find_element(By.PARTIAL_LINK_TEXT, "Sidras").click()
    sleep(5)
    print('Abrindo "Cervejas Estrangeira e Artesanal"')
    element = driver.find_element(By.PARTIAL_LINK_TEXT, "Estrangeira e Artesanal").click()
    sleep(1)
    print('Abrindo "Ver mais produtos"')
    element = driver.find_element(By.LINK_TEXT, "Ver mais produtos").click()
    sleep(1)
    print('Página inteira carregada, passando "element" para o BeautifulSoup4')
    pagina = requests.get(element).text
    soup = BeautifulSoup(pagina, 'html.parser')
except Exception as e:
    print(f'O seguinte erro ocorreu: {e}')
#    driver.close()

In [2]:
# Importing necessary libraries
#import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from time import sleep

# Create a dict with the stores and their website
stores = {'Store':['Continente', 'Pingo Doce'],
          'Website':['https://www.continente.pt', 'https://www.pingodoce.pt']}

# Items that I want to track
my_beers = ['coruja india pale ale', 'guinness', 'franziskaner', 'estrella', 'erdinger', 'benediktiner']
my_choc = ['milka', 'biglicious', 'biiig'] # biglicious e biiig são as barras de 300g do Continente/Pingo Doce

urls_cont = ['https://www.continente.pt/bebidas-e-garrafeira/cervejas-e-sidras/cerveja-estrangeira-e-artesanal/',
            'https://www.continente.pt/bebidas-e-garrafeira/cervejas-e-sidras/cerveja-tradicional/']

choc_cont = ['https://www.continente.pt/mercearia/chocolate-gomas-e-rebucados/chocolates/?start=0&srule=COL-Continente&pmin=0.01']

urls_pingo = ['https://www.pingodoce.pt/on/demandware.store/Sites-pingo-doce-Site/default/Search-Show?cgid=ec_cervejasestrangeiras_1500_200',
            'https://www.pingodoce.pt/on/demandware.store/Sites-pingo-doce-Site/default/Search-Show?cgid=ec_cervejasnacionais_1500_100']

choc_pingo = ['https://www.pingodoce.pt/on/demandware.store/Sites-pingo-doce-Site/default/Search-Show?cgid=ec_tabletes_1200_300_100']


# DAQUI PARA BAIXO TESTADO/FUNCIONA APENAS COM O CONTINENTE
cevas = pd.DataFrame()
chocolates = pd.DataFrame()

# Get the response from the given URLs for beers in Continente.pt
for url in urls_cont:
    response = requests.get(url).text
    soup = BeautifulSoup(response, 'html.parser')

    # With the response, fetch all the available beers names, prices per liter, cost of purchase and packing
    beers = soup.select('h2')
    prices_pl = soup.select('div[class="pwc-tile--price-secondary"]')
    totals = soup.select('span[class="pwc-tile--price-primary"]')
    embalagem = soup.select('p[class="pwc-tile--quantity"]')

    # Creating empty lists to hold site beer names, prices (liter and absolute) and packaging
    nomes = []
    precos_litro = []
    p_total = []
    pack = []

    # Filtering the name of the website product
    try:
        for i in beers:
            nomes.append("Cerveja "+i.get_text().split('com Álcool')[1].lstrip().rstrip())
    except:
        pass

    # Filtering the prices per liter for each product
    for price in prices_pl:
        precos_litro.append(price.get_text().split('€')[0].replace(',', '.').lstrip().rstrip())

    # Filtering the cost to buy each product
    for price in totals:
        p_total.append(price.get_text().split('€')[0].replace(',', '.').lstrip().rstrip())

    # Filtering the product packaging type
    for item in embalagem:
        pack.append(item.get_text().lstrip().rstrip()) 

    ### Creating a DataFrame with all this information
    # Setting column names
    colunas = ['Data', 'Cerveja', 'Embalagem', 'Preço por litro', 'Custo de compra']

    # Date and time as of now
    data = datetime.now().strftime('%d-%m-%Y %H:%M')

    # Joining most lists as a DataFrame
    cervejas = pd.DataFrame([nomes, pack, precos_litro, p_total]).T

    # Concatenating the date and time (just on the first entry)
    cervejas = pd.concat([pd.Series(data), cervejas], ignore_index=True, axis=1)

    # Renaming the columns
    cervejas.columns = colunas

    # Converting prices to float (they're objects until now)
    cervejas['Preço por litro'] = cervejas['Preço por litro'].astype('float')
    cervejas['Custo de compra'] = cervejas['Custo de compra'].astype('float')

    # Filling the date column with, well, the date
    cervejas['Data'] = data

    # Concat?
    cevas = pd.concat([cevas, cervejas], axis=0, ignore_index=True)
    cevas.dropna(inplace=True)
    cevas.drop_duplicates(inplace=True)

for url in choc_cont:
    response = requests.get(url).text
    soup = BeautifulSoup(response, 'html.parser')

    # With the response, fetch all the available beers names, prices per liter, cost of purchase and packing
    chocolate = soup.select('h2')
    prices_pl = soup.select('div[class="pwc-tile--price-secondary"]')
    totals = soup.select('span[class="pwc-tile--price-primary"]')
    embalagem = soup.select('p[class="pwc-tile--quantity"]')

    # Creating empty lists to hold site chocolate names, prices (kg and absolute) and packaging
    nomes = []
    precos_kg = []
    p_total = []
    pack = []

    # Filtering the name of the website product
    for i in chocolate:
        nomes.append(i.get_text().lstrip().rstrip())

    # Filtering the prices per kg for each product
    for price in prices_pl:
        precos_kg.append(price.get_text().split('€')[0].replace(',', '.').lstrip().rstrip())

    # Filtering the cost to buy each product
    for price in totals:
        p_total.append(price.get_text().split('€')[0].replace(',', '.').lstrip().rstrip())

    # Filtering the product packaging type
    for item in embalagem:
        pack.append(item.get_text().lstrip().rstrip())

    ### Creating a DataFrame with all this information
    # Setting column names (the date is already stated before)
    colunas = ['Data', 'Chocolate', 'Embalagem', 'Preço por kg', 'Custo de compra']

    # Joining most lists as a DataFrame
    chocs = pd.DataFrame([nomes, pack, precos_kg, p_total]).T

    # Concatenating the date and time (just on the first entry)
    chocs = pd.concat([pd.Series(data), chocs], ignore_index=True, axis=1)

    # Renaming the columns
    chocs.columns = colunas

    # Converting prices to float (they're objects until now)
    chocs['Preço por kg'] = chocs['Preço por kg'].astype('float')
    chocs['Custo de compra'] = chocs['Custo de compra'].astype('float')

    # Filling the date column with, well, the date
    chocs['Data'] = data

    # Concat?
    chocolates = pd.concat([chocolates, chocs], axis=0, ignore_index=True)
    chocolates.dropna(inplace=True)
    chocolates.drop_duplicates(inplace=True)

In [5]:
print(procura(my_choc, chocolates))
print(procura(my_beers, cevas))

Em 08-02-2026 23:39: Tablete de Chocolate de Leite Milka, emb. 250 gr, por 5.99€, preço por kg 23.96€.
Em 08-02-2026 23:39: Tablete de Chocolate de Leite com Avelãs Inteiras Milka, emb. 250 gr, por 5.99€, preço por kg 23.96€.
Em 08-02-2026 23:39: Tablete de Chocolate com Cheesecake de Morango Milka, emb. 300 gr, por 5.99€, preço por kg 19.97€.
Em 08-02-2026 23:39: Tablete de Chocolate com Amendoim e Caramelo Milka, emb. 276 gr, por 5.99€, preço por kg 21.7€.
Em 08-02-2026 23:39: Tablete de Chocolate Bubbly Caramelo Milka, emb. 250 gr, por 3.99€, preço por kg 15.96€.
Em 08-02-2026 23:39: Tablete de Chocolate de Leite com Bolacha Milka, emb. 300 gr, por 3.99€, preço por kg 13.3€.
Em 08-02-2026 23:39: Tablete de Chocolate de Leite Milka, emb. 90 gr, por 1.99€, preço por kg 22.11€.
Em 08-02-2026 23:39: Tablete de Chocolate Leite Choco & Wafer Milka, emb. 270 gr, por 3.99€, preço por kg 14.78€.
None
Em 08-02-2026 23:39: Cerveja Coruja India Pale Ale Super Bock Coruja, emb. 4 x 33 cl, por 3.